In [1]:
#Importamos las librerias principales
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from pprint import pprint as pp

In [2]:

path="/home/dan/PLATZI/data/Fundamentals/randomforestsalgorithms/data/external"
colum_names=["buying price","maintinance price","number of doors","persons capacity","luggage boot","safty stimated","target"]

#Cargamos dataset a utilizar
df=pd.read_csv(path+"/car_evaluation.csv")
df.columns=colum_names
categorical= df.select_dtypes(include=['object']).columns
uniqueValsSumary={
    col:{'unique count':df[col].nunique(),'unique values':df[col].unique().tolist()}
    for col in categorical
    
}
uniqueValsSumary=pd.DataFrame.from_dict(uniqueValsSumary,orient='index')
uniqueValsSumary


,unique count,unique values
buying price,4,"[vhigh, high, med, low]"
maintinance price,4,"[vhigh, high, med, low]"
number of doors,4,"[2, 3, 4, 5more]"
persons capacity,3,"[2, 4, more]"
luggage boot,3,"[small, med, big]"
safty stimated,3,"[med, high, low]"
target,4,"[unacc, acc, vgood, good]"


In [7]:
#we will separate and train a class tree not having on acount the umbalance
X=df.drop(["target"],axis=1)
Y=df["target"]
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3, random_state=37,stratify=Y)
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import LabelEncoder
encoder=OrdinalEncoder()
labeler=LabelEncoder()
X_train=encoder.fit_transform(X_train)
X_test=encoder.transform(X_test)
Y_train=labeler.fit_transform(Y_train)
Y_test=labeler.transform(Y_test)
# the stratify astribute is necesary because the  encoder for labels does not know what to doo 
# when  in the test set you ask it to transforma a category never seen


from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=5, random_state=73)
rf.fit(X_train,Y_train)
train_predictions=rf.predict(X_train)
predictions=rf.predict(X_test) 
from sklearn.metrics import accuracy_score
train_accuracy=accuracy_score(Y_train,train_predictions)
accuracy=accuracy_score(Y_test,predictions)
print(f"accuracy in trainning: {train_accuracy}, accuracy:{accuracy},feature importance{rf.feature_importances_}")

accuracy in trainning: 0.9942052980132451, accuracy:0.9152215799614644,feature importance[0.22793515 0.15549442 0.06253721 0.18811995 0.07741391 0.28849935]


 the  evaluation metrics appear to good to be true, i will guess there is something wrong because the unbalance data. lets see the clasification report.

In [8]:
from sklearn.metrics import classification_report
print(classification_report(Y_train,train_predictions))

print(classification_report(Y_test,predictions))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       269
           1       1.00      0.98      0.99        48
           2       1.00      1.00      1.00       846
           3       1.00      1.00      1.00        45

    accuracy                           0.99      1208
   macro avg       1.00      0.99      0.99      1208
weighted avg       0.99      0.99      0.99      1208

              precision    recall  f1-score   support

           0       0.80      0.82      0.81       115
           1       0.83      0.71      0.77        21
           2       0.96      0.95      0.96       363
           3       0.83      1.00      0.91        20

    accuracy                           0.92       519
   macro avg       0.86      0.87      0.86       519
weighted avg       0.92      0.92      0.92       519



In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import RandomOverSampler

# Apply oversampling
oversampler = RandomOverSampler(random_state=37)
x, y = oversampler.fit_resample(X, Y)

# Encode features and labels
encoder = OrdinalEncoder()
labeler = LabelEncoder()

# Split the data
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=37, stratify=y
)

# Transform features and labels
x_train = encoder.fit_transform(x_train)
x_test = encoder.transform(x_test)
y_train = labeler.fit_transform(y_train)
y_test = labeler.transform(y_test)

# Train the Random Forest
rf = RandomForestClassifier(n_estimators=5, random_state=73)
rf.fit(x_train, y_train)

# Evaluate the model
train_predictions = rf.predict(x_train)
predictions = rf.predict(x_test)

train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, predictions)

print(f"Accuracy in training: {train_accuracy:.3f}, Accuracy in test: {test_accuracy:.3f}")

# Show feature importances
print("Feature importances:")
feature_names = X.columns if hasattr(X, 'columns') else [f"feature_{i}" for i in range(x_train.shape[1])]
feature_scores = pd.Series(rf.feature_importances_, index=feature_names).sort_values(ascending=False)
print(feature_scores)

# Classification reports
print("\nClassification report on training set:")
print(classification_report(y_train, train_predictions))
print("\nClassification report on test set:")
print(classification_report(y_test, predictions))


Accuracy in training: 0.999, Accuracy in test: 0.988
Feature importances:
safty stimated       0.243092
buying price         0.232674
maintinance price    0.179012
persons capacity     0.153425
luggage boot         0.137946
number of doors      0.053851
dtype: float64

Classification report on training set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       846
           1       1.00      1.00      1.00       846
           2       1.00      1.00      1.00       847
           3       1.00      1.00      1.00       846

    accuracy                           1.00      3385
   macro avg       1.00      1.00      1.00      3385
weighted avg       1.00      1.00      1.00      3385


Classification report on test set:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98       363
           1       1.00      1.00      1.00       363
           2       1.00      0.95      0.97       362
   

the  aparent over fitting of the model can be coused 1 for the use of ordinal encoder in the features, artificial hierarchy , so its important to do this manually  this can be done  use the atibute categories give it an array with the order of the categories for each column  you want to specify the order.